In [1]:
import json
import requests
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random
import glob
import zipfile
import os
import shutil
import csv

In [2]:
class OtpMode(object):
    CAR = 'CAR'
    WALK = 'WALK'
    TRANSIT = 'TRANSIT,WALK'
    BUS = 'BUS,WALK'
    RAIL = 'TRAM,RAIL,SUBWAY,FUNICULAR,GONDOLA,WALK'
    BICYCLE = 'BICYCLE'
    BICYCLECYCLE_TRANSIT = 'TRANSIT,BICYCLE'
    PARK_RIDE = 'CAR_PARK,WALK,TRANSIT'
    KISS_RIDE = 'CAR,WALK,TRANSIT'
    BIKE_RIDE = 'BICYCLE_PARK,WALK,TRANSIT'
    RENTED_BICYCLE = 'WALK,BICYCLE_RENT'
    TRANSIT_RENTED_BICYCLE = 'TRANSIT,WALK,BICYCLE_RENT'
    DRT = 'DRT'
    DRT_TRANSIT = 'DRT_TRANSIT'

    _DICT = ['CAR', 'WALK', 'TRANSIT', 'BUS', 'RAIL', 'BICYCLE', 'BICYCLE_TRANSIT', 'PARK_RIDE', 'KISS_RIDE',
             'BIKE_RIDE', 'RENTED_BICYCLE', 'TRANSIT_RENTED_BICYCLE', 'DRT', 'DRT_TRANSIT']

    _MAIN_MODES = ['CAR', 'BICYCLE', 'TRANSIT', 'WALK']

    _DRT_MODES = ['DRT', 'DRT_TRANSIT']

    _PT_MODES = ['TRANSIT', 'BUS', 'RAIL']

    @staticmethod
    def get_all_modes():
        return [OtpMode.__dict__.get(item) for item in OtpMode._DICT]

    @staticmethod
    def get_main_modes():
        return [OtpMode.__dict__.get(item) for item in OtpMode._MAIN_MODES]

    @staticmethod
    def contains(other):
        return other in OtpMode._DICT

    @staticmethod
    def get_pt_modes():
        return OtpMode._PT_MODES

    @staticmethod
    def get_drt_modes():
        return OtpMode._DRT_MODES

    @staticmethod
    def get_mode(string):
        if OtpMode.contains(string):
            return OtpMode.__getattribute__(OtpMode(), string)
        else:
            raise Exception('unsupported mode {}'.format(string))


In [3]:
class Coord(object):
    """Coordinate.

    Parameters
    ----------
    lat : <float> latitude
    lon : <float> longitude
    latlon : <list> list with both lat and long. Latitude first!
    """
    def __init__(self, lat=None, lon=None, latlon=None):
        if latlon is not None:
            if len(latlon) != 2:
                raise Exception("Wrong coordinate latlon format. Should be a list of two floats.")
            self.lat = latlon[0]
            self.lon = latlon[1]
        elif lat is None or lon is None:
            raise Exception("Coordinates not provided")
        else:
            self.lat = lat
            self.lon = lon

    def to_json(self):
        return json.dumps(self, default=lambda o: self._try(o), sort_keys=True, indent=4, separators=(',', ':'))

    @staticmethod
    def _try(o):
        try:
            if o.__class__ == Coord:
                raise Exception()
            return o.__dict__
        except:
            return str(o)
        
    def __str__(self):
        return str(self.lat) + ',' + str(self.lon)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.lat == other.lat and self.lon == other.lon

    def __hash__(self):
        return hash((self.lat, self.lon))


In [4]:
class Leg(object):
    """Leg of a trip. For example, "walk - bus - walk" trip has three legs.
    Used to store trip legs from OTP.

    Parameters
    ----------
    mode : <str> mode of transport
    start_coord : <coord> coordinate of an origin
    end_coord : <coord> coordinate of a destination
    distance : <int> meters
    duration : <int> seconds
    steps : <list> of utils.Step
    """

    # TODO:assignment of mode   as a string is confusing, remove it, or use constant
    def __init__(self, mode=None, start_coord=None, from_stop=None, end_coord=None, to_stop=None,
                 start_time=None, end_time=None,
                 distance=None, duration=None, steps=None):
        self.mode = mode
        self.start_coord = start_coord
        self.end_coord = end_coord
        self.distance = distance
        self.duration = duration
        self.steps = steps
        # The two below only used for PT legs
        self.from_stop = from_stop
        self.to_stop = to_stop

        self.start_time = start_time
        self.end_time = end_time

    def deepcopy(self):
        if self.steps is None:
            steps = []
        else:
            steps = [step.deepcopy() for step in self.steps if step is not None]
        return Leg(mode=copy.copy(self.mode),
                   start_coord=copy.copy(self.start_coord),
                   from_stop=copy.copy(self.from_stop),
                   end_coord=copy.copy(self.end_coord),
                   to_stop=copy.copy(self.to_stop),
                   start_time=copy.copy(self.start_time),
                   end_time=copy.copy(self.end_time),
                   distance=copy.copy(self.distance),
                   duration=copy.copy(self.duration),
                   steps=steps)


In [5]:
class Trip(object):
    """A list of legs and a total trip duration
    """
    legs = ...  # type: List[Leg]

    def __init__(self):
        self.legs = []
        self.duration = None
        self.distance = None
        self.main_mode = None

    def set_empty_trip(self, mode, coord_start, coord_end):
        """Sets a dummy trip between two coordinates with zero distance, duration and one empty leg"""
        self.set_duration(0)
        self.set_distance(0)
        self.main_mode = mode
        self.legs = [Leg(mode=mode, start_coord=coord_start, end_coord=coord_end, distance=0, duration=0,
                         steps=[Step(coord_start, coord_end, 0, 0)])]

    def dumps(self):
        return self.__dict__

    def get_leg_modes(self):
        """Returns a list of modes from the legs"""
        return [l.mode for l in self.legs]

    def deepcopy(self):
        nt = Trip()
        nt.duration = copy.copy(self.duration)
        nt.distance = copy.copy(self.distance)
        nt.main_mode = copy.copy(self.main_mode)
        nt.legs = [leg.deepcopy() for leg in self.legs]
        return nt

    def main_mode_from_legs(self):
        leg_modes = self.get_leg_modes()

        if LegMode.CAR in leg_modes:
            return OtpMode.CAR
        elif LegMode.BUS in leg_modes or LegMode.SUBWAY in leg_modes or \
                LegMode.TRAM in leg_modes or LegMode.RAIL in leg_modes:
            return OtpMode.TRANSIT
        elif LegMode.BICYCLE in leg_modes:
            return OtpMode.BICYCLE
        elif LegMode.WALK in leg_modes:
            return OtpMode.BICYCLE
        else:
            log.error('Main mode unrecognized. Returning None. Kick the developer to make a proper function.')
            return None

    def set_duration(self, dur):
        self.duration = dur

    def set_main_mode(self, mode):
        self.main_mode = mode
    
    def set_distance(self, dist):
        self.distance = dist
    
    def append_leg(self, leg):
        self.legs.append(leg)
        
    def __str__(self):
        return '{} trip, takes {} distance {}'\
            .format(self.main_mode, self.duration, self.distance)

    def __repr__(self):
        return str(self)


In [6]:
class Step(object):
    """Arguments:|
    start_coord       <Coord>|
    distance    <int>|
    duration    <int>|
    """
    def __init__(self, start_coord, end_coord, distance, duration):
        self.start_coord = start_coord
        self.end_coord = end_coord
        self.distance = distance
        self.duration = duration

    @staticmethod
    def get_empty_step(coord):
        return Step(start_coord=coord, end_coord=coord, distance=0, duration=0)

    def deepcopy(self):
        return Step(start_coord=copy.copy(self.start_coord),
                    end_coord=copy.copy(self.end_coord),
                    distance=copy.copy(self.distance),
                    duration=copy.copy(self.duration),
                    )

    def dumps(self):
        return self.__dict__

    def __str__(self):
        return 'Step distance {:.1f}, duration {:.1f}'.format(self.distance, self.duration)

    def __repr__(self):
        return self.__str__()


In [7]:
def osrm_route_request(from_place, to_place):
    url_coords = '{}{},{};{},{}' \
        .format('http://0.0.0.0:5000/route/v1/driving/',
                from_place.get('lon'), from_place.get('lat'), to_place.get('lon'), to_place.get('lat'))
    url_full = url_coords + '?annotations=true&geometries=geojson&steps=true'
    resp = requests.get(url=url_full)
    return _parse_osrm_response(resp)


def _parse_osrm_response(resp):
    # if resp.status_code != requests.codes.ok:
    #     resp.raise_for_status()

    jresp = resp.json()
    # if jresp.get('code') != 'Ok':
    #     log.error(jresp.get('code'))
    #     log.error(jresp.get('message'))
    #     resp.raise_for_status()

    trip = Trip()
    trip.legs = [Leg()]
    trip.legs[0].steps = []

    legs = jresp.get('routes')[0].get('legs')
    for leg in legs:
        steps = leg.get('steps')
        for step in steps:
            new_step = Step(distance=step.get('distance'),
                            duration=step.get('duration'),
                            start_coord=Coord(lon=step.get('geometry').get('coordinates')[0][0],
                                              lat=step.get('geometry').get('coordinates')[0][1]),
                            end_coord=Coord(lon=step.get('geometry').get('coordinates')[-1][0],
                                            lat=step.get('geometry').get('coordinates')[-1][1]))
            # OSRM makes circles on roundabouts. And makes empty step in the end. Exclude these cases from a route
            if new_step.start_coord != new_step.end_coord:
                trip.legs[0].steps.append(new_step)
        if len(trip.legs[0].steps) == 0:
            waypoints = jresp.get('waypoints')
            trip.legs[0].steps.append(Step(distance=0,
                                           duration=0,
                                           start_coord=Coord(lon=waypoints[0].get('location')[0],
                                                             lat=waypoints[0].get('location')[1]),
                                           end_coord=Coord(lon=waypoints[1].get('location')[0],
                                                           lat=waypoints[1].get('location')[1])
                                           )
                                      )
    trip.legs[0].start_coord = trip.legs[0].steps[0].start_coord
    trip.legs[0].end_coord = trip.legs[0].steps[-1].end_coord
    trip.legs[0].duration = sum([step.duration for step in trip.legs[0].steps])
    trip.legs[0].distance = sum([step.distance for step in trip.legs[0].steps])
    trip.legs[0].mode = OtpMode.DRT

    trip.distance = trip.legs[0].distance
    trip.duration = trip.legs[0].duration
    trip.main_mode = OtpMode.CAR
    return trip


In [28]:
scenario_names = ['0','1','2','3','4']
path_to_zip = '/home/ai6644/Malmo/Tools/Simulation results/Lolland/TRB/drt rerun/dtm1.1/'

In [29]:
direct_min= []
direct_km = []
for scenario_name in scenario_names:
# scenario_name = "4"
    print(scenario_name)
    
    path_to_zip_file = path_to_zip + scenario_name+'/log.zip'

    day_sec = int(86400)
    day_h = int(24)

    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        try:
            shutil.rmtree('data/zip')
        except (FileNotFoundError, OSError) as e:
            pass
        os.mkdir('data/zip')
        zip_ref.extractall('data/zip')
    path = 'data/zip/' + os.listdir('data/zip')[0] + '/trip_dump.json'

    all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_occupancy_*")
    n_cars = len(all_files)
    
    fp = open(path)
    tdump = json.load(fp)
    fp.close()
    
    persons = tdump.get('person')
    
    drt_legs = []
    for person in persons:
        trips = person.get('actual_trips')
        for trip in trips:
                legs = [leg for leg in trip.get('legs') if leg.get('mode') in ['DRT', 'DRT_TRANSIT']]
                if len(legs) > 0:    
                    drt_legs += legs
    #                 direct_legs += [leg.get('duration') for leg in act.get('legs') if leg.get('mode') in ['DRT', 'DRT_TRANSIT']])
                
    direct_seconds_drt_only = sum([osrm_route_request(leg.get('start_coord'), leg.get('end_coord')).duration for leg in drt_legs])
    direct_minutes = direct_seconds_drt_only / 60
    print('direct_minutes' ,str(direct_minutes).replace('.',','))
    direct_min.append(str(direct_minutes).replace('.',','))
    
    direct_vkt = sum([osrm_route_request(leg.get('start_coord'), leg.get('end_coord')).distance for leg in drt_legs])
    print('direct kilommetres' ,str(direct_vkt/1000).replace('.',','))
    direct_km.append(str(direct_vkt/1000).replace('.',','))
print('direct km')
for a in direct_km:
    print(a)
print('direct_min')
for a in direct_min:
    print(a)
    
print(';'.join(['direct km'] + direct_km))
print(';'.join(['direct_min'] + direct_min))

0
direct_minutes 11961,076666666673
direct kilommetres 9632,720099999995
1
direct_minutes 12087,869999999995
direct kilommetres 9792,158900000008
2
direct_minutes 12388,574999999953
direct kilommetres 10097,180100000016
3
direct_minutes 11930,073333333325
direct kilommetres 9624,0061
4
direct_minutes 11582,783333333316
direct kilommetres 9379,938600000001
direct km
9632,720099999995
9792,158900000008
10097,180100000016
9624,0061
9379,938600000001
direct_min
11961,076666666673
12087,869999999995
12388,574999999953
11930,073333333325
11582,783333333316
direct km;9632,720099999995;9792,158900000008;10097,180100000016;9624,0061;9379,938600000001
direct_min;11961,076666666673;12087,869999999995;12388,574999999953;11930,073333333325;11582,783333333316


In [30]:
n_vehicles = []
active_hours = []
cost1 = []
for scenario_name in scenario_names:
# scenario_name = "4"
    print(scenario_name)
    
    path_to_zip_file = path_to_zip + scenario_name+'/log.zip'

    day_sec = int(86400)
    day_h = int(24)
    
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        try:
            shutil.rmtree('data/zip')
        except (FileNotFoundError, OSError) as e:
            pass
        os.mkdir('data/zip')
        zip_ref.extractall('data/zip')
    path = 'data/zip/' + os.listdir('data/zip')[0] + '/vehicle_logs'

    all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_occupancy_*")
    all_files.sort()

    occupancies = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df['time'] = df.time.astype(int)
        df = df.drop_duplicates(keep='last', subset='time')
        occupancies.append(df)

    merged_time = pd.concat(occupancies)['time'].sort_values().drop_duplicates()
    
    n_cars = len(all_files)
    
    all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_status_*")
    all_files.sort()

    statuses = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        df.columns = ['time', 'status']
        df['time'] = df.time.astype(int)
        df = df.drop_duplicates(keep='last', subset='time')
        statuses.append(df)
        
    times = []
    for vid, status in enumerate(statuses):
        t_idle = 0
        t_wait = 0
        t_drive = 0
        t_return = 0
        t_stop = 0
        started = False # not used, but we may use it to add empty waiting time to idle time
        for (_, st1), (_,st2) in zip(status.iterrows(), status.iloc[1:].iterrows()):

            t = st2.time - st1.time

            # departed
            if st1.status == 6 and st2.status != 6:
                started = True
            # staying at depot
            if st1.status == 6:
                t_idle += t
            # arrived to depot
            if st1.status != 6 and st2.status == 6:
                started = False

            if st1.status == 4:
                t_wait += t

            # driving
            if st1.status == 3:
                t_drive += t

            # returning
            if st1.status == 5:
                t_return += t

            #on a stop
            if st1.status in (0,2):
                t_stop +=t

        if len(status) == 1:
            t_idle = 86400
        else:
            t_idle += 86400 - st2.time

        if (t_idle + t_wait + t_drive + t_return + t_stop != 86400):
            print('somehing is wrong!')
            break

        times.append(pd.DataFrame([t_idle, t_wait, t_drive, t_return, t_stop], index=['idle', 'wait', 'drive', 'return', 'stop'], columns=[vid]).T)
        
    vehicle_times = pd.concat(times)
    vehicle_times = vehicle_times[vehicle_times.idle != 86400]
    
#     **** Number of vehicles from chart ************
    all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_occupancy_*")
    all_files.sort()

    occupancies = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df['time'] = df.time.astype(int)
        df = df.drop_duplicates(keep='last', subset='time')
        occupancies.append(df)

    merged_time = pd.concat(occupancies)['time'].sort_values().drop_duplicates()
    n_cars = len(all_files)
    
    all_files = glob.glob("data/zip/*/vehicle_logs/vehicle_status_*")
    all_files.sort()

    statuses = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        df.columns = ['time', 'status']
        df['time'] = df.time.astype(int)
        df = df.drop_duplicates(keep='last', subset='time')
        statuses.append(df)

    for occ,stat in zip(occupancies, statuses):
        occ.set_index('time', inplace=True)
        stat.set_index('time', inplace=True)
        # idle
        occ['#passengers'].loc[(stat.status == 6).values] = -2
        # waiting outside depot
        occ['#passengers'].loc[(stat.status == 4).values] = -1

    merged_df = pd.concat(occupancies, axis=1).fillna(method='ffill')
    
    li=[]
    for i in range(0, len(merged_df)):
        li.append(merged_df.iloc[i].value_counts())
    #     li[-1].index = 8 - li[-1].index.values    
    
    day_values = pd.concat(li, axis=1).fillna(0).T.reindex(index=[_ for _ in range(0, day_sec)]).fillna(method='ffill')
    day_values = day_values[day_values.columns.to_list()[::-1]]
    day_values.columns = day_values.columns.to_list()[:-2] + ['waiting', 'idling']
    day_values2 = day_values.copy()
    day_values2['idling'] = day_values2.idling + day_values2.waiting
    day_values2 = day_values2.drop(['waiting'], axis=1)
    number_vehicles = n_cars - day_values2.idling.min()
# *******************
    
    print('number of vehicles', str(number_vehicles))
    n_vehicles.append(str(number_vehicles).replace('.',','))
    a = (vehicle_times.drive.sum() + vehicle_times["return"].sum() + vehicle_times.stop.sum())/3600
    print('vehicle_times', str(a).replace('.',','))
    active_hours.append(str(a).replace('.',','))
    
    # waiting time is partially prised
    cost = (vehicle_times.drive + vehicle_times.stop + vehicle_times['return']).sum()*(367/60/60) + \
        ((vehicle_times.idle).sum() + vehicle_times.wait.sum() - len(vehicle_times)*9*60*60 - (len(vehicle_times) - number_vehicles)*60*60*15)*(367/60/60)*0.7
    print('cost', str(cost).replace('.',','))
    cost1.append(str(cost).replace('.',','))
    
print('n_vehicles')
for a in n_vehicles:
    print(a)
print('active_hours')
for a in active_hours:
    print(a)
print('cost1')
for a in cost1:
    print(a)
    
print(';'.join(['n_vehicles'] + n_vehicles))
print(';'.join(['active_hours'] + active_hours))
print(';'.join(['cost1'] + cost1))

0
number of vehicles 27.0
vehicle_times 241,54222222222222
cost 130638,29866666665
1
number of vehicles 27.0
vehicle_times 245,15333333333334
cost 131035,88199999998
2
number of vehicles 31.0
vehicle_times 253,2888888888889
cost 147345,60666666663
3
number of vehicles 32.0
vehicle_times 248,295
cost 150649,27949999998
4
number of vehicles 28.0
vehicle_times 232,66583333333332
cost 133514,50824999998
n_vehicles
27,0
27,0
31,0
32,0
28,0
active_hours
241,54222222222222
245,15333333333334
253,2888888888889
248,295
232,66583333333332
cost1
130638,29866666665
131035,88199999998
147345,60666666663
150649,27949999998
133514,50824999998
n_vehicles;27,0;27,0;31,0;32,0;28,0
active_hours;241,54222222222222;245,15333333333334;253,2888888888889;248,295;232,66583333333332
cost1;130638,29866666665;131035,88199999998;147345,60666666663;150649,27949999998;133514,50824999998


In [31]:
total = []
executed = []
excluded = []
car = []
transit = []
walk = []
drt = []
drt_transit = []
vkt = []

for scenario_name in scenario_names:
    print(scenario_name)
    
    path_to_zip_file = path_to_zip + scenario_name+'/log.zip'

    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        try:
            shutil.rmtree('data/zip')
        except (FileNotFoundError, OSError) as e:
            pass
        os.mkdir('data/zip')
        zip_ref.extractall('data/zip')

    fp = open('data/zip/' + os.listdir('data/zip')[0] + '/log_summary.log')
    lines = fp.readlines()
    fp.close()
        
    mode_share = True
    vkt_not_filled = True
    
    for line in lines:
        line = line.rstrip()
        if "Leg share :" in line:
            mode_share = False
        
        if "Total " in line:
            total.append(line.split(' ')[1])
        elif "Executed trips:" in line:
            executed.append(line.split(' ')[2])
        elif "Excluded persons due to none or a trivial path" in line:
            excluded.append(line.split(' ')[9])
        elif "CAR " in line and mode_share:
            car.append(line.split(' ')[1])
        elif "TRANSIT,WALK " in line and mode_share:
            transit.append(line.split(' ')[1])
        elif "WALK " in line and mode_share:
            walk.append(line.split(' ')[1])
        elif "DRT_trips " in line and mode_share:
            drt.append(line.split(' ')[1])
        elif "DRT_TRANSIT_trips " in line and mode_share:
            drt_transit.append(line.split(' ')[1])
        elif "Vehicle kilometers " in line and vkt_not_filled:
            vkt.append(line.split(' ')[2])
            vkt_not_filled = False
        
    
print(";".join(['total']+[a for a in total]))
print(";".join(['executed']+[a for a in executed]))
print(";".join(['excluded']+[a for a in excluded]))
print(";".join(['car']+[a for a in car]))
print(";".join(['transit']+[a for a in transit]))
print(";".join(['walk']+[a for a in walk]))
print(";".join(['drt']+[a for a in drt]))
print(";".join(['drt_transit']+[a for a in drt_transit]))

print(";".join(['vkt']+[str(a) for a in vkt]))

0
1
2
3
4
total;1246;1257;1265;1235;1242
executed;1245;1255;1264;1234;1241
excluded;1;2;1;1;1
car;101;104;93;87;111
transit;3;1;2;1;1
walk;0;0;0;0;0
drt;1007;1011;1048;1014;1013
drt_transit;134;139;121;132;116
vkt;9614.102100000002;9829.536099999998;10249.8941;9955.697699999999;9327.694200000002


In [32]:
print(";".join(['total']+[a for a in total]))
print(";".join(['executed']+[a for a in executed]))
print(";".join(['excluded']+[a for a in excluded]))
print(";".join(['car']+[a for a in car]))
print(";".join(['transit']+[a for a in transit]))
print(";".join(['walk']+[a for a in walk]))
print(";".join(['drt']+[a for a in drt]))
print(";".join(['drt_transit']+[a for a in drt_transit]))
print(";".join(['vkt']+[str(a) for a in vkt]))

print(';'.join(['direct km'] + [a.replace(',','.') for a in direct_km]))
print(';'.join(['direct_min'] + [a.replace(',','.') for a in direct_min]))
print(';'.join(['n_vehicles'] + [a.replace(',','.') for a in n_vehicles]))
print(';'.join(['active_hours'] + [a.replace(',','.') for a in active_hours]))
print(';'.join(['cost1'] + [a.replace(',','.') for a in cost1]))

total;1246;1257;1265;1235;1242
executed;1245;1255;1264;1234;1241
excluded;1;2;1;1;1
car;101;104;93;87;111
transit;3;1;2;1;1
walk;0;0;0;0;0
drt;1007;1011;1048;1014;1013
drt_transit;134;139;121;132;116
vkt;9614.102100000002;9829.536099999998;10249.8941;9955.697699999999;9327.694200000002
direct km;9632.720099999995;9792.158900000008;10097.180100000016;9624.0061;9379.938600000001
direct_min;11961.076666666673;12087.869999999995;12388.574999999953;11930.073333333325;11582.783333333316
n_vehicles;27.0;27.0;31.0;32.0;28.0
active_hours;241.54222222222222;245.15333333333334;253.2888888888889;248.295;232.66583333333332
cost1;130638.29866666665;131035.88199999998;147345.60666666663;150649.27949999998;133514.50824999998
